In [2]:
from pyaspeller import YandexSpeller, Word
from spellchecker import SpellChecker
from tqdm import tqdm
from datetime import datetime
import pandas as pd

speller = YandexSpeller()
russian = SpellChecker(language='ru')

In [3]:
# тестовые данные

data = pd.read_csv('data\prepr_set.csv')

In [2]:
s = 'В суббботу утромь. Кружок3 летел в верх насонце в месте скаровой также в метро смотрель ехал че3рт. В суббботу утромь. Кружок3 летел в верх насонце в месте скаровой также в метро смотрель ехал че3рт. В суббботу утромь. Кружок3 летел в верх насонце в месте скаровой также в метро смотрель ехал че3рт. В суббботу утромь. Кружок3 летел в верх насонце в месте скаровой также в метро смотрель ехал че3рт. В суббботу утромь. Кружок3 летел в верх насонце в месте скаровой также в метро смотрель ехал че3рт. В суббботу утромь. Кружок3 летел в верх насонце в месте скаровой также в метро смотрель ехал че3рт.'
s = 'Иванов'
ss = ''


start = datetime.now()

for w in tqdm(s.split(' ')):
    check = Word(w)
    if not check.correct:
        w = check.spellsafe
    ss += w + ' '
    
end_time = datetime.now() - start
print('Time yandex spellchecker {}'.format(end_time))
print(ss)

start = datetime.now()

for w in tqdm(s.split(' ')):
    correction_form = russian.correction(w)

    if correction_form != w:
        w = correction_form
    ss += w + ' '

end_time = datetime.now() - start
print('Time pyspellchecker {}'.format(end_time))
print(ss)

# ss

  0%|          | 0/1 [00:00<?, ?it/s]c:\program files\anaconda3\envs\venv_bb\lib\site-packages\pyaspeller\word.py:17: UserWarning: Class Word is deprecated. Use Speller().spelled(text)
  warnings.warn("Class Word is deprecated. Use Speller().spelled(text)")
100%|██████████| 1/1 [00:00<00:00, 124.90it/s]

Time yandex spellchecker 0:00:00.959530
Иванов 
Time pyspellchecker 0:00:00.030391
Иванов иванова 


In [3]:
fixed = speller.spelled(s)
fixed

'Иванов'

In [4]:
from nlp_functions.nlp_functions import nlp_preprocessing
from nlp_functions.tf_idf_preprocessing import tf_idf

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
# 1 
# проверка месяцев
txt1 = '1) Тексст 2) Опечатком 3)Смастирили лофт в нутри небаскребаа 4) поросоль 5) смотрибельно 6) чуства 7)ещё 8)шооу маст гооу онн'
txt2 = '1)Никита  3)Москва 4)Иванов 5)Тридцать 6)Восемь 7)пять с кредиторами обычно 9)'
txt3 = '1)Января 2) марта 3)апреля'

df = pd.DataFrame(data=[txt1, txt2, txt3], columns=['text'])

In [7]:
# 2
df = data.iloc[:2000]['txt'].to_frame()
df.columns = ['text']

### Препроцессинг текста

In [8]:
nlp_params = dict()

## Основные параметры
nlp_params['train'] = df # основной датасет
nlp_params['oos'] = None  # out-of-sample при наличии (необязательный параметр)
nlp_params['oot'] = None  # out-of-time при наличии (необязательный параметр)
nlp_params['text_field'] = 'text'  # название столбца с текстом

## Простой препроцессинг
nlp_params['need_del_dash'] = True  # удаление тире
nlp_params['need_lower_case'] = True  # приведение к нижнему регистру
nlp_params['need_del_number'] = False  # удаление чисел
nlp_params['need_del_in_brackets'] = False  # внутри скобок
nlp_params['need_del_eng'] = True  # английские буквы

## Поиск опечаток (spellcheker)
nlp_params['need_spellchecker'] = True  # запуск поиска опечаток [True, False]
# действия над опечатками [delete, replace, nothing(default)] / рекомендуется использовать replace - работает в разы быстрее
nlp_params['need_del_spell'] = 'replace'

## Лемматизация
nlp_params['need_lemma'] = True  # запуск лемматизации [True, False]
nlp_params['need_lru_cache'] = True  # использование декоратора для ускорения процесса лемматизации [True, False] (рекомендуется)

## Поиск сущностей (NER) | поиск сущностей следует запускать только после лемматизации слов
nlp_params['need_ner'] = True  # запуск поиска сущностей [True, False]
nlp_params['need_del_number_ner'] = 'replace'  # действия над числами [delete, replace, nothing(default)]
nlp_params['need_del_name'] = 'replace'  # действия над именами (также)
nlp_params['need_del_org'] = 'replace'  # действия над названиями организаций (также)
nlp_params['need_del_geo'] = 'replace'  # действия над локациями (также)
nlp_params['need_del_months'] = True  # действия над месяцами (True - delete, False - nothing)

## Настройки для стопслов
nlp_params['need_del_stopwords'] = True  # действия над стопсловами [True, False]
nlp_params['new_stopwords'] = []  # новый список стопслов ['word1', 'word2']
# режим работы со стопсловами, какие стопслова используются ['default list', 'default list + additional list', 'only additional list']
nlp_params['mode_stopwords'] = 'default list'  

train, oos, oot = nlp_preprocessing(**nlp_params)

Trash chars done! - 0:00:00.144399

Progress train lemmatization/ner/spellchecker: 2000 / 2000

Lemmatization / Spellcheker / NER done! - 0:12:56.974667

Delete stop words done! - 0:00:00.464223
Preprocessing done!


In [9]:
train

,text,new_prep_text
0,"При ликвидации, реорганизации или уменьшении у...",ликвидация реорганизация уменьшение уставный ...
1,Предоставить дополнительное обеспечение либо п...,предоставить дополнительный обеспечение либо п...
2,"Ежеквартально не позднее ""10""-го числа месяца,...",ежеквартально поздний NUMBER число месяц следу...
3,"Заемщик обязан в сроки, указанные в соответств...",заемщик обязать срок указанный соответствующий...
4,В течение 5 (Пяти) рабочих дней с Даты доставк...,течение NUMBER рабочий день дата доставка уве...
...,...,...
1995,Заемщик обязан застраховать (обеспечить страхо...,заемщик обязать застраховать обеспечить страхо...
1996,6.2.9 Договора читать в следующей редакции:\n«...,NUMBER договор читать следующий редакция пред...
1997,«Предоставить дополнительное обеспечение либо ...,предоставить дополнительный обеспечение либо п...
1998,"Заемщик обязан застраховать имущество, передав...",заемщик обязать застраховать имущество передав...


### Векторизация текста с помощью TFiDF

In [10]:
tf_idf_params = dict()

params = { # Параметры TFidfVectorizer()
    'ngram_range': (1, 1) # задаем размер н-грамм
}

## Основные параметры
tf_idf_params['train'] = train # основной датасет
tf_idf_params['oos'] = None  # out-of-sample при наличии (необязательный параметр)
tf_idf_params['oot'] = None  # out-of-time при наличии (необязательный параметр)
tf_idf_params['text_field'] = 'new_prep_text'  # название столбца с текстом
tf_idf_params['target_name'] = None  # название поля с таргетом

## Параметры TFiDF
tf_idf_params['params'] = params

train, oos, oot = tf_idf(**tf_idf_params)

In [11]:
train

,0,1,2,3,4,5,6,7,8,9,...,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
